In [ ]:
import os
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np

models = tf.keras.models
layers = tf.keras.layers

batch_size = 32
img_height = 800
img_width = 1200

In [ ]:
def get_image_list(baseFolder: str):
    genres = os.listdir(baseFolder)
    print(f"Genres: {genres}")
    image_list = []
    for genre in genres:
        genre_folder = os.path.join(baseFolder, genre)
        for image in os.listdir(genre_folder):
            if not image.endswith('.png'):
                print(f"Skipping {image}, not a png")
                continue
            image_list.append((os.path.join(genre_folder, image), genre))
    return image_list


def adjust_dataset_size(imageList):
    genre_count = {}
    np.random.shuffle(imageList)
    for image, genre in imageList:
        if genre not in genre_count:
            genre_count[genre] = 0
        genre_count[genre] += 1
    lowest_count = min(genre_count.values())
    new_image_list = []
    for image, genre in imageList:
        if genre_count[genre] > lowest_count:
            genre_count[genre] -= 1
            continue
        new_image_list.append((image, genre))
    return imageList


def make_dataset(baseFolder: str):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        baseFolder,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    val_ds = tf.keras.utils.image_dataset_from_directory(
        baseFolder,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        
        batch_size=batch_size)
    
    normalization_layer = layers.Rescaling(1./255)
    normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    return train_ds, val_ds


In [ ]:
def make_model():
    model = models.Sequential([
        layers.experimental.preprocessing.Rescaling(1. / 255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2)
    ])
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
                  loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


def train_model(model, X_train, y_train, X_val, y_val):
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=3
    )
    return model


def evaluate_model(model, val_ds):
    loss, accuracy = model.evaluate(val_ds)
    return loss, accuracy

In [ ]:
def main():
    print(f"running from {os.getcwd()}")
    baseFolder = '../data/spectrograms/'
    train, validation = make_dataset(baseFolder)
    X_train, y_train = train
    X_test, y_test = validation
    model = make_model()
    model = train_model(model, X_train, y_train, X_test, y_test)
    loss, accuracy = evaluate_model(model, X_test, y_test)
    print(f'Loss: {loss}, Accuracy: {accuracy}')


if __name__ == '__main__':
    main()